In [ ]:
import pandas as pd
import cv2
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from sklearn.metrics import f1_score
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

In [ ]:
physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
main_folder = "C:/Users/ahmet/Calismalarim/opencv"
image_folder = main_folder + "/img_align_celeba/img_align_celeba/"
example_picture = "C:/Users/ahmet/Calismalarim/opencv/img_align_celeba/img_align_celeba/000103.jpg"

In [ ]:
df_ = pd.read_csv(main_folder + '/list_attr_celeba.csv')
df = df_.copy()
df.set_index('image_id', inplace=True)
df.replace(to_replace=-1, value=0, inplace=True)
df.shape

In [ ]:
for i, j in enumerate(df.columns):
    print(i, j)

In [ ]:
df.rename(columns={"Male": "Gender"}, inplace=True)

In [ ]:
img = load_img(example_picture)
plt.grid(False)
plt.imshow(img)

df.loc[example_picture.split('/')[-1]][['Attractive','Gender','Young']]

In [ ]:
plt.title("Female and Male")
sns.countplot(y="Gender", data=df, color="r")

In [ ]:
df_part = pd.read_csv(main_folder + "/list_eval_partition.csv")
df_part.head()

In [ ]:
df_part.partition.value_counts()

# 0 -> train
# 1 -> validation
# 2 -> test

In [ ]:
df_part.set_index("image_id", inplace=True)
df_part = df_part.join(df.Gender, how='inner')
df_part.head()

In [ ]:
img_width = 178
img_height = 218

def load_reshape_img(fname):
    img = load_img(fname)
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    
    df_ = df_part[(df_part['partition'] == partition) 
                           & (df_part[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_part[(df_part['partition'] == partition) 
                                  & (df_part[attr] == 1)].sample(int(num_samples/2))])

    if partition != 2:
        x_ = np.array([load_reshape_img(image_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 218, 178, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(image_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (img_width, img_height)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,      
    width_shift_range=0.2,  
    height_shift_range=0.2,
    shear_range=0.2,       
    zoom_range=0.2,      
    horizontal_flip=True,  
)

In [ ]:
img = load_img(example_picture)
x = img_to_array(img)/255.
x = x.reshape((1,) + x.shape)

plt.figure(figsize=(20,10))
plt.suptitle('Data Augmentation', fontsize=28)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.subplot(3, 5, i+1)
    plt.grid(False)
    plt.imshow( batch.reshape(218, 178, 3))
    
    if i == 9:
        break
    i += 1

In [ ]:
training_samples = 20000
x_train, y_train = generate_df(0, 'Gender', training_samples)

train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=32,
)

In [ ]:
validation_samples=4000
x_valid, y_valid = generate_df(1, 'Gender', validation_samples)

valid_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

valid_datagen.fit(x_valid)

valid_generator = valid_datagen.flow(
x_valid, y_valid,
batch_size=32,
)

In [ ]:
inception_model = InceptionV3(weights='C:\Users\user\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                              include_top=False, input_shape = (img_height, img_width, 3))

print("number of layers:", len(inception_model.layers))

In [ ]:
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

custom_model = Model(inputs=inception_model.input, outputs=predictions)

In [ ]:
for layer in custom_model.layers[:52]:
    layer.trainable = False

custom_model.compile(optimizer=Adam(learning_rate=0.0005)
                    , loss='categorical_crossentropy'
                    , metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=1)

hist = custom_model.fit(train_generator, validation_data=(x_valid, y_valid), 
                 steps_per_epoch=training_samples/32, epochs=20, verbose=1, callbacks = [early_stopping])

In [ ]:
def visualize_training(history, lw = 3):
    plt.figure(figsize=(10,10))
    plt.subplot(2,1,1)
    plt.plot(history.history['accuracy'], label = 'training', marker = '*', linewidth = lw)
    plt.plot(history.history['val_accuracy'], label = 'validation', marker = 'o', linewidth = lw)
    plt.title('Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend(fontsize = 'x-large')
    
visualize_training(hist)

In [ ]:
test_samples = 4000
x_test, y_test = generate_df(2, 'Gender', test_samples)
model_predictions = [np.argmax(custom_model.predict(feature)) for feature in x_test ]

test_accuracy = 100 * np.sum(np.array(model_predictions)==y_test) / len(model_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)
print('f1_score:', f1_score(y_test, model_predictions))

In [ ]:
gender_target = {0: 'Female'
                , 1: 'Male'}

def img_to_display(filename):
 
    i = Image.open(filename)
    i.thumbnail((200, 200), Image.LANCZOS)
    
    with BytesIO() as buffer:
        i.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()
    

def display_result(filename, prediction, target):
    gender = 'Male'
        
    if prediction[1] <= 0.5:
        gender = 'Female'
            
    display_html = '''
    <div style="overflow: auto;  border: 2px solid #D8D8D8;
        padding: 5px; width: 480px;" >
        <img src="data:image/jpeg;base64,{}" style="float: left;" width="200" height="200">
        <div style="padding: 10px 0px 0px 20px; overflow: auto;">
            <h3 style="margin-left: 50px; margin-top: 2px;">Prediction: {}</h3>
            <p style="margin-left: 50px; margin-top: -6px; font-size: 12px">{} probability</p>
            <p style="margin-left: 50px; margin-top: -16px; font-size: 12px">Real gender: {}</p>
        </div>
    </div>
    '''.format(img_to_display(filename)
               , gender
               , "{0:.2f}%".format(round(max(prediction)*100,2))
               , gender_target[target]
               , filename.split('/')[-1]
               )

    display(HTML(display_html))

In [ ]:
def gender_prediction(filename):
    im = cv2.imread(filename)
    im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (178, 218)).astype(np.float32) / 255.0
    im = np.expand_dims(im, axis=0)
    
    result = custom_model.predict(im)
    prediction = np.argmax(result)
    
    return result

df_to_test = df_part[df_part['partition'] == 2].sample(3)

for index, target in df_to_test.iterrows():
    result = gender_prediction(image_folder + index)
    display_result(image_folder + index, result[0], target['Gender'])
